# Ejemplo basico de analisis de datos en streaming con Spark

En este ejemplo vamos a usar Structured Streaming de spark que es la version mas actualizada de procesamiento en microlotes provista por Spark. 

En primer lugar es necesario correr el script stream-to-socket.py en python en una ventana de un terminal aparte. Este script se va a encargar de enviar un stream continuo de datos hacia el puerto 9999 con una linea del texto test.txt cada 5 segundos. Luego el script en PySpark va a leer de ese stream y publicar una funcion de agregacion que cuenta las palabras por cada microlote.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Crear la sesión de Spark
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

# Crear DataFrame representando el stream de entrada desde la conexión a localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Dividir las líneas en palabras
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generar conteo de palabras en tiempo real
wordCounts = words.groupBy("word").count()

# Función para mostrar cada microbatch en el notebook
def show_batch(df, epoch_id):
    df.show(truncate=False)

# Iniciar la consulta
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .foreachBatch(show_batch) \
    .start()

# Esperar la terminación de la consulta
query.awaitTermination()


+----+-----+
|word|count|
+----+-----+
+----+-----+

+------------------+-----+
|word              |count|
+------------------+-----+
|GutenbergÃ¢â‚¬â„¢s|1    |
|AliceÃ¢â‚¬â„¢s    |1    |
|by                |1    |
|for               |1    |
|in                |1    |
|is                |1    |
|Lewis             |1    |
|use               |1    |
|Wonderland        |1    |
|the               |1    |
|Adventures        |1    |
|eBook             |1    |
|Carroll           |1    |
|This              |1    |
|Project           |1    |
+------------------+-----+

+--------------+-----+
|word          |count|
+--------------+-----+
|anyone        |1    |
|AliceÃ¢â‚¬â„¢s|2    |
|by            |2    |
|in            |2    |
|with          |1    |
|Lewis         |2    |
|cost          |1    |
|Wonderland    |2    |
|anywhere      |1    |
|Adventures    |2    |
|and           |1    |
|of            |1    |
|no            |1    |
|at            |1    |
|Carroll       |2    |
+--------------+---

# Ejercicios propuestos
- Agregar un filtro para que solo tome palabras que comienzan con mayusculas
- Modificar la salida para que en cada batch, solo muestre los casos donde hubo un cambio en la cuenta de palabras.